# 1. Colab 환경설정


In [ ]:
! python --version

Python 3.9.16


In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.0 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp39-cp39-linux_x86_64.whl size=680535 sha256=6706b302fb4eed4aa269f82b127233b36ac35b85457f910196f09b6d2e368d0b
  Stored in directory: /root/.cache/pip/wheels/47/17/70/b257bc53879a458c4bfcc900e89271aa8b4f19366a54bd2455
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-n626g_gz
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-n626g_gz
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 2. 데이터셋 전처리

In [ ]:
import pandas as pd
chatbot_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/GDSC_voicephishiing/conversation_data_set.xlsx')

In [ ]:
chatbot_data.sample(n=10)

,Unnamed: 0,0,1
1310,1310,근데 그 해수욕장은 너무 재밌는 게 뭐냐면 그 온천 물과 그 바다가 같이 공존을 해...,0
1245,1245,저는 최근에 몬스타엑스라는 연예인에 빠져있는데요. 몬스타엑스는 일곱 명으로 구성되어...,0
1899,1899,길거리를 가다가 떨어져 있는 낙엽을 보든 나무를 보든 정말 같은 빨간색이 하나도 없...,0
271,271,우체국 위에는 제가 그 제일 많이 쓰거든요. 사용을 제일 많이 하거든요. 뭐 입금도...,1
1067,1067,2년 전 손학규 바른미래당 대표 시절에 현 문재인 대통령에 대한 평가 기사인 듯합니...,0
625,625,네. 네 그러다 보니까 이제 대한민국에서 본인 조사할 수 있는 사람은 저밖에 없어요...,1
367,367,음 이게 이런 부분에 대해서 금융감독원칙으로 기재 처리를 해서 결과 보고서 올려 드...,1
251,251,되게 정부 지원 상품이라서 제가 이제 조금 월권 행사를 할 수 있는 부분인데 고객님...,1
1866,1866,근데 아기 얘들이 사료를 주니까 하루 아침저녁 이렇게 사료를 주니까 자기들끼리 개체...,0
1100,1100,내가 중학교 2학년 때 미국을 갔다 왔잖아. 막 시험 보고 이렇게 해가지고 진짜 2...,0


In [ ]:

data_list = []
for q, label in zip(chatbot_data.iloc[:,1], chatbot_data.iloc[:,2])  :
    data = []
    data.append(q)
    data.append(str(int(label)))

    data_list.append(data)

In [ ]:
print(data_list[0])
print(data_list[400])
print(data_list[800])
print(data_list[1200])
print(data_list[1600])
print(data_list[2000])
print(data_list[-1])

['여보세요 응 이런 사람은 알고 계세요. 건물이나 돈을 범죄 사겠다는 권고하는 과정에서 대량의 신용카드를 대포통령이 발견이 됐는데 그중 문제가 되는게 거기서 지금 명의로 된 하나은행 통장과 농협 통장에 발견이 됐어요. 그 두 개의 통장 개설에 대해서 아시는 내용 있으세요 아니요? 전혀 없어요음 그 두 개의 통장을 작년 3월 3일 경기도 광명시 철산동에서 바둑 계절을 개설된 계좌인데 어 전혀 모르셨어요음네 알겠습니다. 끝없이 통화 불편하시면 말씀을 해 주세요음 제출 계좌를 불법 현장에서 알 수 있기 때문에 동결 처리와 동시에 북구로 환수 조치했고요. 금융 운동은 제주공 결과 또 계좌번호도 확실히 명의 계정이한테 저희가 확인을 했어요. 제가 어 저에게 연락드린 이유는 어? 본인이 직접 통장을 개설하셔서 일당들에게 금전적인 대가를 받고 통장을 판매하신 건 아닌지 일단 아니면은 제 개인 정보가 유출이 되면서 명의 조정 당하실 피해자 분이신지에 대해서 저는 연락을 드린 거예요. 통장을 판매는 양봉하신 적 있으세요 아니요? 통장을 판매를 양도 하시는 것도 아니라고 보이는 하셨고도 모른다고 보기는 하셨는데 그러면은 사이에 지갑이나 핸드폰이랑 개인정보가 있을 만한 물건을 분실하신 데가 있을까요 아니요 없어요음 질문을 좀 똑바로 좀 해주세요 장난치지 마시고. 장난 아닌데요. 네, 알겠습니다. 자 그러면은 아 혹시 이런 쇼핑몰 사이트를 이용하세요 11번가나 옥션 지마켓. 라면 그냥 저희가 이걸 왜 못 잡았냐면 대부분 평소들이 이런 식품으로 사이트에서 개인 정보였지 상당히 많이 됐더라고요. 안녕! 네 저 지금 이거 조사를 어 한 두 시간 정도 할 건데 여기 괜찮으세요? 자.', '1']
['서울중앙지방검찰청에서 전화드린 거예요네네 자 현재 저희 검찰에서 김용식 주범으로 인한 금융범죄 사기단의 검거한 상태예요. 자이 검거 현장에서 대량의 신용카드가 대포통장을 압수하던 과정에 연기로 돼 이제 농협 하나는 통장 두 점이 발견되어 조석제 연락드린 거고요. 자 문의 혹시이 통장 개별 거

# 3. Train data와 Test data로 데이터 나누기

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:

print(len(dataset_train))
print(len(dataset_test))

1645
549


# 4. KoBERT 입력 데이터로 만들기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# 5. KoBERT 학습모델 만들기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

# 6. KoBERT 모델 학습시키기

In [ ]:
# for e in range(num_epochs):
#     train_acc = 0.0
#     test_acc = 0.0
#     model.train()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
#         optimizer.zero_grad()
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         loss = loss_fn(out, label)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
#         optimizer.step()
#         scheduler.step()  # Update learning rate schedule
#         train_acc += calc_accuracy(out, label)
#         if batch_id % log_interval == 0:
#             print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
#     print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
#     model.eval()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         test_acc += calc_accuracy(out, label)
#     print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
# torch.save(model, "/content/drive/MyDrive/voice_phishing/conversation_model.pt")

FileNotFoundError: ignored

# 7. 새로운 문장 테스트

In [ ]:
import numpy as np

In [ ]:
# model_sent = torch.load("/content/drive/MyDrive/voice_phishing/sentence_model.pt")

In [ ]:
#CPU 사용
device = torch.device("cpu")

In [ ]:
model_conv = torch.load("/content/drive/MyDrive/Colab Notebooks/GDSC_voicephishiing/conversation_model.pt", map_location=device)

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model_conv.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model_conv(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            print(logits)
            if np.argmax(logits) == 0:
                test_eval.append("일반 대화")
            elif np.argmax(logits) == 1:
                test_eval.append("보이스피싱")
            result = softmax(logits)
            
    result_list = result.tolist()
    # result_list.sort()
    # print(result_list)
  

    # print(f">> {result_list[-1]*100}% 확률로 " + test_eval[0] + "입니다.")
    return result_list

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
!pip install flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip ls

ERROR: unknown command "ls" - maybe you meant "list"


In [ ]:
from flask import Flask, jsonify, request, Response

In [ ]:
app = Flask(__name__)

@app.route('/classify/')
def classify():
    # Handle GET request here
    # sentence = request.get_json()
    print("request recieved!")
    try:
      # print("handle...")
      data = {'message': 'Hello, world!'}
      # print(request.get_json())
      sentence = request.get_json()
      # print("data achieved!")
      # print(sentence)
      # print(sentence["data"])
      data = predict(sentence["data"])
      print("predict success!")
      # print(sentence)
    except Exception as e:
      print(f" Err: {e}")
      data = {'message': 'Hello, world!'}

    
    # data = {'message': 'Hello, world!'}
    return jsonify(data[1])

In [ ]:
# Start ngrok tunnel
from pyngrok import ngrok
public_url = ngrok.connect(port=80).public_url
print("Public URL:", public_url)

Public URL: http://c105-35-184-51-255.ngrok.io


In [ ]:
app.run(port=80)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:80
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Mar/2023 09:01:20] "GET /classify HTTP/1.1" 308 -


request recieved!
handle...
{'data': ' 아 서울 중앙 지검 첨단 범죄 수사 일부 이희나 수사관입니다. 본 사건 조사는 법원 증거 제출용으로 이뤄지고 있으신데 녹취 조사 과정에서 허위 진술이나 본인께서 아는 내용을 은닉하게되면은 본인은 이 사건과 무관하게 위증과 공무 집행 방해 죄로 가중 처벌되실 수가 있으시고 민  형사상 책임이 있으시니 사실에 입각하여 진실로 좀 하셔야 됩니다. 네 네 OOO씨 지금 제가 본인 확인 한번 할께요. 생년월일 앞에 여섯 자리만 말씀 부탁 드리겠습니다. 네 OOOOOO 네 성함은 OOO님 맞으시죠? 네 네 본인 김성오 일당 잘 알고 있으십니까? 아니요 전혀 모른다고 진술 하셨는데 맞습니까? 네 네 이천 십 칠년이 월 삼 일 경기도 광명시 철산동 지점에서 본인의 명의로 되어 있는 농협과 하나 은행 두개 통장이 계설 된 것으로 확인이 되었는데 본인께서 해당 금융권 직접 내방하셔서 계설하셨습니까? 아니요 이 두개 계좌 계설 사실에 대해서 전혀 모르신다는 말씀 이시죠? 네 네 본인은 네이버 중고나라 카페 이용해 주시고 있나요? 네 네 이 사건은 전자 금융 거래법 위반  금융 범죄 사기 사건으로 다수의 피해자가 발생한 OOO 고소 고발 사건입니다. 네 다만 OOO님도 개인 정보 유출로 인해 명의 도용 피해자 일 수 있기 때문에 저희가 재산권 보호 차원에서 계좌 동결 전에 금융 감독원 공주하에 본인 계좌 추적을 통해서 다른 불법적인 계좌가 있으신 지 김성오 일당과 금전 거래가 있는지 확인을 좀 할 것입니다. 네 근데 이런 조사 과정에서 불법적으로 사용된 내용이 발견이 되신다면 본인 명의의 금융 계좌 동결과 함께 불법 자금은 국고로 환수 조치 되실 거에요. 네 현재 OOO씨 본인 명의로 거래하시는 은행과 기타 금융권 계좌 개설 사실에 대해서 본인께서 직접 내방을 해서 개설하신 거 있잖아요? 네 네 본인께서 아시는 내용만 말씀해 주시면 되는데 현제 본인 명의로 되어 있는 계좌가 어느 금융권에 몇 개를 이용하고 계시죠

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO:werkzeug:127.0.0.1 - - [12/Mar/2023 09:01:21] "GET /classify/ HTTP/1.1" 200 -


[-3.5021605  3.749207 ]
predict success!


...

request recieved!
handle...
 Err: 400 Bad Request: The browser (or proxy) sent a request that this server could not understand.


INFO:werkzeug:127.0.0.1 - - [12/Mar/2023 09:02:29] "GET /classify HTTP/1.1" 308 -


request recieved!
handle...
{'data': ' 아 서울 중앙 지검 첨단 범죄 수사 일부 이희나 수사관입니다. 본 사건 조사는 법원 증거 제출용으로 이뤄지고 있으신데 녹취 조사 과정에서 허위 진술이나 본인께서 아는 내용을 은닉하게되면은 본인은 이 사건과 무관하게 위증과 공무 집행 방해 죄로 가중 처벌되실 수가 있으시고 민  형사상 책임이 있으시니 사실에 입각하여 진실로 좀 하셔야 됩니다. 네 네 OOO씨 지금 제가 본인 확인 한번 할께요. 생년월일 앞에 여섯 자리만 말씀 부탁 드리겠습니다. 네 OOOOOO 네 성함은 OOO님 맞으시죠? 네 네 본인 김성오 일당 잘 알고 있으십니까? 아니요 전혀 모른다고 진술 하셨는데 맞습니까? 네 네 이천 십 칠년이 월 삼 일 경기도 광명시 철산동 지점에서 본인의 명의로 되어 있는 농협과 하나 은행 두개 통장이 계설 된 것으로 확인이 되었는데 본인께서 해당 금융권 직접 내방하셔서 계설하셨습니까? 아니요 이 두개 계좌 계설 사실에 대해서 전혀 모르신다는 말씀 이시죠? 네 네 본인은 네이버 중고나라 카페 이용해 주시고 있나요? 네 네 이 사건은 전자 금융 거래법 위반  금융 범죄 사기 사건으로 다수의 피해자가 발생한 OOO 고소 고발 사건입니다. 네 다만 OOO님도 개인 정보 유출로 인해 명의 도용 피해자 일 수 있기 때문에 저희가 재산권 보호 차원에서 계좌 동결 전에 금융 감독원 공주하에 본인 계좌 추적을 통해서 다른 불법적인 계좌가 있으신 지 김성오 일당과 금전 거래가 있는지 확인을 좀 할 것입니다. 네 근데 이런 조사 과정에서 불법적으로 사용된 내용이 발견이 되신다면 본인 명의의 금융 계좌 동결과 함께 불법 자금은 국고로 환수 조치 되실 거에요. 네 현재 OOO씨 본인 명의로 거래하시는 은행과 기타 금융권 계좌 개설 사실에 대해서 본인께서 직접 내방을 해서 개설하신 거 있잖아요? 네 네 본인께서 아시는 내용만 말씀해 주시면 되는데 현제 본인 명의로 되어 있는 계좌가 어느 금융권에 몇 개를 이용하고 계시죠

INFO:werkzeug:127.0.0.1 - - [12/Mar/2023 09:02:31] "GET /classify/ HTTP/1.1" 200 -


predict success!


In [ ]:
# Import necessary libraries
import socket
import requests

# Get the public IP address of the Colab runtime
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.connect(("8.8.8.8", 80))
public_ip_address = s.getsockname()[0]
s.close()

# Print the IP address
print("Public IP address of the Colab runtime:", public_ip_address)

Public IP address of the Colab runtime: 172.28.0.12


In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=1a6f166ccee2012757c580c8af9fabab46e7c620f0399e8da001b3aa686935ae
  Stored in directory: /root/.cache/pip/wheels/f6/89/59/49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok


In [ ]:
!ngrok authtoken 2Bd65SUazYWfMaAm9a2LhqSRps8_4Seeutz6NQiD7v1S3PU2T

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    result_list = predict(sentence)
    print(result_list[-1])
    print("\n")

하고싶은 말을 입력해주세요 : "아 서울 중앙 지검 첨단 범죄 수사 일부 이희나 수사관입니다. 본 사건 조사는 법원 증거 제출용으로 이뤄지고 있으신데 녹취 조사 과정에서 허위 진술이나 본인께서 아는 내용을 은닉하게되면은 본인은 이 사건과 무관하게 위증과 공무 집행 방해 죄로 가중 처벌되실 수가 있으시고 민  형사상 책임이 있으시니 사실에 입각하여 진실로 좀 하셔야 됩니다." 네 네 OOO씨 지금 제가 본인 확인 한번 할께요. 생년월일 앞에 여섯 자리만 말씀 부탁 드리겠습니다. 네 OOOOOO 네 성함은 OOO님 맞으시죠? 네 네 본인 김성오 일당 잘 알고 있으십니까? 아니요 전혀 모른다고 진술 하셨는데 맞습니까? 네 네 이천 십 칠년이 월 삼 일 경기도 광명시 철산동 지점에서 본인의 명의로 되어 있는 농협과 하나 은행 두개 통장이 계설 된 것으로 확인이 되었는데 본인께서 해당 금융권 직접 내방하셔서 계설하셨습니까? 아니요 이 두개 계좌 계설 사실에 대해서 전혀 모르신다는 말씀 이시죠? 네 네 본인은 네이버 중고나라 카페 이용해 주시고 있나요? 네 "네 이 사건은 전자 금융 거래법 위반  금융 범죄 사기 사건으로 다수의 피해자가 발생한 OOO 고소 고발 사건입니다." 네 다만 OOO님도 개인 정보 유출로 인해 명의 도용 피해자 일 수 있기 때문에 저희가 재산권 보호 차원에서 계좌 동결 전에 금융 감독원 공주하에 본인 계좌 추적을 통해서 다른 불법적인 계좌가 있으신 지 김성오 일당과 금전 거래가 있는지 확인을 좀 할 것입니다. 네 근데 이런 조사 과정에서 불법적으로 사용된 내용이 발견이 되신다면 본인 명의의 금융 계좌 동결과 함께 불법 자금은 국고로 환수 조치 되실 거에요. 네 현재 OOO씨 본인 명의로 거래하시는 은행과 기타 금융권 계좌 개설 사실에 대해서 본인께서 직접 내방을 해서 개설하신 거 있잖아요? 네 네 본인께서 아시는 내용만 말씀해 주시면 되는데 현제 본인 명의로 되어 있는 계좌가 어느 금융권에 몇 개를 이용하고 계시죠? 국민 은행에.. 국민 은행에.

KeyboardInterrupt: ignored

In [ ]:
"아 서울 중앙 지검 첨단 범죄 수사 일부 이희나 수사관입니다. 본 사건 조사는 법원 증거 제출용으로 이뤄지고 있으신데 녹취 조사 과정에서 허위 진술이나 본인께서 아는 내용을 은닉하게되면은 본인은 이 사건과 무관하게 위증과 공무 집행 방해 죄로 가중 처벌되실 수가 있으시고 민, 형사상 책임이 있으시니 사실에 입각하여 진실로 좀 하셔야 됩니다.",네,네 OOO씨 지금 제가 본인 확인 한번 할께요. 생년월일 앞에 여섯 자리만 말씀 부탁 드리겠습니다.,네 OOOOOO,네 성함은 OOO님 맞으시죠?,네,네 본인 김성오 일당 잘 알고 있으십니까?,아니요,전혀 모른다고 진술 하셨는데 맞습니까?,네,네 이천 십 칠년이 월 삼 일 경기도 광명시 철산동 지점에서 본인의 명의로 되어 있는 농협과 하나 은행 두개 통장이 계설 된 것으로 확인이 되었는데 본인께서 해당 금융권 직접 내방하셔서 계설하셨습니까?,아니요,이 두개 계좌 계설 사실에 대해서 전혀 모르신다는 말씀 이시죠?,네,네 본인은 네이버 중고나라 카페 이용해 주시고 있나요?,네,"네 이 사건은 전자 금융 거래법 위반, 금융 범죄 사기 사건으로 다수의 피해자가 발생한 OOO 고소 고발 사건입니다.",네,다만 OOO님도 개인 정보 유출로 인해 명의 도용 피해자 일 수 있기 때문에 저희가 재산권 보호 차원에서 계좌 동결 전에 금융 감독원 공주하에 본인 계좌 추적을 통해서 다른 불법적인 계좌가 있으신 지 김성오 일당과 금전 거래가 있는지 확인을 좀 할 것입니다.,네,근데 이런 조사 과정에서 불법적으로 사용된 내용이 발견이 되신다면 본인 명의의 금융 계좌 동결과 함께 불법 자금은 국고로 환수 조치 되실 거에요.,네,현재 OOO씨 본인 명의로 거래하시는 은행과 기타 금융권 계좌 개설 사실에 대해서 본인께서 직접 내방을 해서 개설하신 거 있잖아요?,네,네 본인께서 아시는 내용만 말씀해 주시면 되는데 현제 본인 명의로 되어 있는 계좌가 어느 금융권에 몇 개를 이용하고 계시죠?,국민 은행에.. 국민 은행에.. 휴먼 계좌도 다 얘기 해야 하나요?,예 예 예,"국민은행에 하나 사용 